# Greenland, 2010-2019, Max: 205, Min: 305

In [ ]:
from proposed.query_executor_timeseries import TimeseriesExecutor

# Latitude: 59.79 to 83.63, approximately 25 degrees
# Longitude: -73.00 to -12.00, approximately 60 degrees

qe = TimeseriesExecutor(
    variable="2m_temperature",
    start_datetime="2010-01-01 00:00:00",
    end_datetime="2019-12-31 23:00:00",
    temporal_resolution="hour",
    max_lat = 85,
    min_lat = 60,
    min_lon = -70,
    max_lon = -10,
    spatial_resolution=0.25,
    aggregation="mean",
    time_series_aggregation_method="mean",
)
ts1 = qe.execute()
ts1

In [ ]:
ts1.max()

In [ ]:
ts1.min()

In [ ]:
import sys
sys.path.append('vanilla/')

from vanilla.vanilla_get_timeseries_executor import VanillaGetTimeseriesExecutor

vanilla_qe = VanillaGetTimeseriesExecutor(
    variable="2m_temperature",
    start_datetime="2010-01-01 00:00:00",
    end_datetime="2019-12-31 23:00:00",
    temporal_resolution="hour",
    max_lat = 85,
    min_lat = 60,
    min_lon = -70,
    max_lon = -10,
    spatial_resolution=0.25,
    aggregation="mean",
)
vts = vanilla_qe.execute()


In [ ]:
vts

In [ ]:
ts1.max()

In [ ]:
ts1.min()

In [ ]:
import sys
sys.path.append('proposed/')
from proposed.query_executor_find_time import FindTimeExecutor

fqe = FindTimeExecutor(
    variable="2m_temperature",
    start_datetime="2010-01-01 00:00:00",
    end_datetime="2019-12-31 23:00:00",
    temporal_resolution="hour",
    max_lat = 85,
    min_lat = 60,
    min_lon = -75,
    max_lon = -10,
    spatial_resolution=0.25,
    aggregation="mean",
    time_series_aggregation_method="mean",
    filter_predicate=">",
    filter_value=255,
)
ft = fqe.execute()
ft
    

In [ ]:
import sys
sys.path.append('vanilla/')

from vanilla.vanilla_get_timeseries_executor import VanillaGetTimeseriesExecutor

vanilla_qe = VanillaGetTimeseriesExecutor(
    variable="2m_temperature",
    start_datetime="2010-01-01 00:00:00",
    end_datetime="2010-12-31 23:00:00",
    temporal_resolution="hour",
    max_lat = 85,
    min_lat = 60,
    min_lon = -75,
    max_lon = -10,
    spatial_resolution=0.25,
    aggregation="max",
)
vts = vanilla_qe.execute()

In [ ]:
vts.min(), vts.max()

In [ ]:
import sys
sys.path.append('proposed/')
from proposed.query_executor_find_time import FindTimeExecutor

fqe = FindTimeExecutor(
    variable="2m_temperature",
    start_datetime="2010-01-01 00:00:00",
    end_datetime="2010-12-31 23:00:00",
    temporal_resolution="hour",
    max_lat = 85,
    min_lat = 60,
    min_lon = -75,
    max_lon = -10,
    spatial_resolution=0.25,
    aggregation="min",
    time_series_aggregation_method="min",
    filter_predicate=">",
    filter_value=240,
)
ft = fqe.execute()
ft
    

In [1]:
from proposed.query_executor_timeseries import TimeseriesExecutor

# Latitude: 59.79 to 83.63, approximately 25 degrees
# Longitude: -73.00 to -12.00, approximately 60 degrees

qe = TimeseriesExecutor(
    variable="2m_temperature",
    start_datetime="2004-01-01 00:00:00",
    end_datetime="2023-12-31 23:00:00",
    temporal_resolution="hour",
    max_lat = 85,
    min_lat = 60,
    min_lon = -70,
    max_lon = -10,
    spatial_resolution=0.25,
    aggregation="min",
    time_series_aggregation_method="min",
)
ts1 = qe.execute()
print(ts1.min())

qe = TimeseriesExecutor(
    variable="2m_temperature",
    start_datetime="2004-01-01 00:00:00",
    end_datetime="2023-12-31 23:00:00",
    temporal_resolution="hour",
    max_lat = 85,
    min_lat = 60,
    min_lon = -70,
    max_lon = -10,
    spatial_resolution=0.25,
    aggregation="max",
    time_series_aggregation_method="max",
)
ts1 = qe.execute()
print(ts1.max())

local files: ['/data/era5/agg/2m_temperature/2m_temperature-hour_100.nc']
<xarray.Dataset> Size: 4B
Dimensions:  ()
Data variables:
    t2m      float32 4B 209.6
local files: ['/data/era5/agg/2m_temperature/2m_temperature-hour_100.nc']
<xarray.Dataset> Size: 4B
Dimensions:  ()
Data variables:
    t2m      float32 4B 303.5
